In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.activations import relu
from tensorflow.keras import Model, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Dropout
import pandas as pd

In [ ]:
print(tf.config.list_physical_devices('GPU'))
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], enable = True)

In [ ]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001
EPOCHS = 5

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_images = os.getcwd() + "/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [ ]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
input_layer = Input(shape=(64,64,1), name='input_layer')
conv1 = Conv2D(32,3, kernel_regularizer = regularizers.l2(WEIGHT_DECAY))(input_layer, name = 'conv1')
batchnorm1= BatchNormalization()(conv1)
act1 = relu(batchnorm1)
maxpool1 = MaxPool2D((2,2))(act1)

conv2 = Con2D(64 3, kernel_regularizers = regularizers.l2(WEIGHT_DECAY))
batchnorm2 = BatchNormalization()(conv2)
act2 = relu(batchnorm2)
maxpool2 = MaxPool2D((2,2))(act2)

conv3 = Conv2D(128, activation='relu', kernel_regularizers = regularizers.l2(WEIGHT_DECAY))(maxpool2)
maxpool3 = MaxPool2D((2,2))(conv3)
flatten = Flatten()(maxpool3)
dense1 = Dense(128, activation='relu')(flatten)
dropout = Dropout(0.5)(dense1)
dense2 = Dense(128, activation='relu')(dropout)
output1 = Dense(10, activation = 'softmax', name='first_num')(dense2)
output2 = Dense(10, activation='softmax', name='second_num')(dense2)
      
model = Model(inputs = [input_layer], outputs = [output1, output2])
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = tf.keras.optimizers.Adam(lr = 0.01), metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
model.fit(train_dataset, batch_size = BATCH_SIZE, epochs= EPOCHS, verbose = 2)
model.evaluate(test_dataset, batch_size = BATCH_SIZE, verbose=0)